In [1]:
import numpy as np
import pandas as pd
import datetime
import torch.optim as optim
from scipy import io
import argparse
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import h5py  
import matplotlib.pyplot as plt
import matplotlib
# print(matplotlib.__version__)
matplotlib.use('Agg')
import h5py

In [2]:
from PIL import Image
import math
from sklearn.metrics import confusion_matrix
import pylab as pl
import numpy as np
import itertools
import torchvision

In [3]:

current_file_data_save = 'C:/Users/Asus/Documents/DL-Partial_fourier/Save/'
data_file = 'C:/Users/Asus/Documents/DL-Partial_fourier/calgary_multicoil/'
# data_file = 'C:/Users/Asus/Documents/DL-Partial_fourier/singlecoil_test/'

In [4]:
os.makedirs(current_file_data_save+'/test/', exist_ok=True)  
os.makedirs(current_file_data_save+'/validation/', exist_ok=True)
os.makedirs(current_file_data_save+'/train/', exist_ok=True)
matrix_size = int(218)

In [5]:
pf_line = int(np.floor(matrix_size*0.45))
pf_line_com = matrix_size-pf_line
pf_line_o =int(np.floor(170*0.45)) 
SS_flag = 0
MS_flag = 0
EMS_flag = 1

In [6]:
filePath =data_file 
filename = os.listdir(filePath)
length = len(filename)
print(length)

47


In [7]:
for idx in range(0,15):   # change 15 to length to run for all
    f = h5py.File(filePath+ filename[idx],'r')
    
    kspace = f['kspace']
    kspace = np.transpose(kspace,(0, 2, 1, 3))
    kspace_shape = kspace.shape
    
    if kspace_shape[1] == 170:    # K-space has 170 and 180 but we work on 170 only
        print(f.keys())
        data_R = kspace[:,:,:,0:12];
        data_I = kspace[:,:,:,12:24];


        data = data_I*1j +data_R
        sz = data.shape
        
        
        freq= np.fft.ifftshift(data ,axes=0)
        freq = np.fft.ifft(freq,axis =0)
        data1 = np.sqrt(sz[0])*np.fft.fftshift(freq,axes=0)
        data2 = np.reshape(data1,(-1,1,sz[1],sz[2]))

        freq= np.fft.ifftshift(data2 ,axes=(2,3))
        freq = np.fft.ifft2(freq,axes=(2,3))
        img = np.sqrt(sz[1]*sz[2])*np.fft.fftshift(freq,axes=(2,3))

        scale_ref = round(np.max(np.absolute(img)),15)
        img = img/scale_ref

        ##single slice
        if SS_flag ==1:
            img_ref  = img[1:sz[0]-1,:,:,:]*0
            for k in range(1,sz[0]-1):
                img_ref[k-1,:,:,:] = img[k,:,:,:]
        else:
            img_ref = np.concatenate((img[1:sz[0]-1,:,:,:]*0,img[1:sz[0]-1,:,:,:]*0,img[1:sz[0]-1,:,:,:]*0),axis = 1)
            for k in range(1,sz[0]-1):
                img_ref[k-1,:,:,:] = np.concatenate((img[k,:,:,:],  img[k-1,:,:,:], img[k+1,:,:,:]), axis =0) 
                
                
#         num_zeros = np.count_nonzero(img_ref == 0)
#         print("Number of zeros in the array:", num_zeros)

        ##multi slice
        freq= np.fft.ifftshift(img_ref ,axes=(2,3))
        freq= np.fft.fft2(freq ,axes=(2,3))
        
        test_k= 1/np.sqrt(sz[1]*sz[2])*np.fft.fftshift(freq,axes=(2,3))   
        test_data = np.copy(test_k)
        test_data[:,0,:,:pf_line] = 0

        test_data[:,0,:pf_line_o,:] = 0    
    ##############complementry sampling pattern
        if EMS_flag ==1:
            test_data[:,1:3,:,pf_line_com:] = 0

        if MS_flag==1:
            test_data[:,1:3,:,:pf_line] = 0 
    ########################### 

        freq = np.fft.ifftshift(test_data,axes=(2,3))
        freq = np.fft.ifft2(freq ,axes=(2,3))
        img_data = np.sqrt(sz[1]*sz[2])*np.fft.fftshift(freq,axes=(2,3))   

        img_label =img_ref[:,0,:,:] -img_data[:,0,:,:]
        img_label =img_label.reshape(sz[0]-2,-1,sz[1],sz[2])

        imgdataR = np.copy(np.real(img_data))
        imgdataI = np.copy(np.imag(img_data))

        imgdata = np.concatenate((imgdataR,imgdataI), axis=1)



        imglabelR = np.copy(np.real(img_label))
        imglabelI = np.copy(np.imag(img_label))
        imglabel = np.concatenate((imglabelR,imglabelI), axis=1)


        imgfull=img_label[:,0,:,:] + img_data[:,0,:,:]
        imgfull =imgfull.reshape(sz[0]-2,-1,sz[1],sz[2])

        for i in range(16,int(sz[0]-20)):
            D = torch.from_numpy(imgdata[i,:,:,:]).float()
            L = torch.from_numpy(imglabel[i,:,:,:]).float()
            data = {'k-space':D,'label':L}
            torch.save(data,current_file_data_save +'/train/'+str(idx)+'_'+str(i)+'.pth')
        f.close()

<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
<KeysViewHDF5 ['kspace']>
Number of zeros in the array: 0
